# loading

In [1]:
import os
import struct
import random
import cv2
#import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,'%s-labels-idx1-ubyte'% kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte'% kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',lbpath.read(8))
        labels = np.fromfile(lbpath,dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",imgpath.read(16))
        images = np.fromfile(imgpath,np.uint8).reshape(len(labels), 784)

    return images, labels

#### Loading the data

X_train, y_train = load_mnist('./data', kind='newtrain')
X_test, y_test = load_mnist('./data', kind='new1k')

In [2]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    images_path = os.path.join(path,
                               '%s-patterns-idx3-ubyte'
                               % kind)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",imgpath.read(16))
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(60000, 784)

    return images

X_testall = load_mnist('./data', kind='testall')

In [3]:
print('train_sample_number:\t:%d, column_number:%d' %(X_train.shape[0], X_train.shape[1]))
print('test_sample_number:\t:%d, column_number:%d' %(X_test.shape[0], X_test.shape[1]))
print('testall_sample_number\t:%d, column_number:%d' %(X_testall.shape[0], X_testall.shape[1]))

train_sample_number:	:80000, column_number:784
test_sample_number:	:10000, column_number:784
testall_sample_number	:60000, column_number:784


In [5]:
#X_train=X_train/255
X_test=X_test/255
#X_testall=X_testall/255

# deskew

In [6]:
from scipy.ndimage import interpolation

def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix

In [7]:
def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    return interpolation.affine_transform(image,affine,offset=offset)

In [8]:
def deskewAll(X):
    currents = []
    for i in range(len(X)):
        currents.append(deskew(X[i].reshape(28,28)).flatten())
    return np.array(currents)

In [9]:
#X_train_deskewed = deskewAll(X_train)
#X_test_deskewed = deskewAll(X_test)
X_test=deskewAll(X_test)
#np.shape(X_train_deskewed)

https://github.com/fsix/mnist/blob/master/notebooks/Deskewing.ipynb

# hog

In [10]:
from skimage.feature import hog

def calc_hog_features(X, image_shape=(28, 28), pixels_per_cell=(8, 8)):
    fd_list = []
    for row in X:
        img = row.reshape(image_shape)
        fd = hog(img, orientations=8, pixels_per_cell=pixels_per_cell, cells_per_block=(1, 1))
        fd_list.append(fd)
    
    return np.array(fd_list)

In [11]:
#hoged_X_train = calc_hog_features(X_train, pixels_per_cell=(8, 8))
hoged_X_test = calc_hog_features(X_test, pixels_per_cell=(8, 8))
hoged_X_fea=hoged_X_test.shape[1]
#np.shape(hoged_X_test)

https://www.kaggle.com/njanakiev/mnist-hog

# convolution

In [12]:
kenel = [
    [1,0,-1],
    [1,0,-1],
    [1,0,-1]
]

In [13]:
def convolution(X,kenel):
    con_X =np.zeros((X.shape[0],26*26))
    for y in range(1,26):
        for x in range(1,26):
            temp=X[y-1:y+2,x-1:x+2]
            mul_=np.dot(temp,kenel)
            sum_=np.sum(mul_)
            con_X[y-1][x-1]=sum_
    return con_X

In [14]:
#con_X_train=convolution(X_train,kenel)
con_X_test=convolution(X_test,kenel)
con_X_fea=con_X_test.shape[1]
#np.shape(con_X_test)

# preprocessing

In [15]:
#X_train=np.hstack((X_train,con_X_train,hoged_X_train))#,con_X_train))
X_test=np.hstack((X_test,con_X_test,hoged_X_test))

# classifer

In [16]:
class myClassifier(object):    
    """
    ovr
    """
    def __init__(self, C=1000, eta=0.01, batch_size=60, epochs=200, epsilon=1e-8, 
                 shuffle=True, params=None, w=0, b=0):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.epsilon = epsilon
        self.class_num = 0
        self.shuffle = shuffle
        self.update_count = 0
        self.w = 0
        self.b = 0
        
    def fit(self, X, y, params=None, w=0, b=0, testscore = None, eval_score=None):
        self.class_num=len(np.unique(y))
        #self.class_num=len(cp.unique(y))
        
        if params is None:
            print('fit params=None')
            self.params = {
                'w': np.random.randn(X.shape[1], self.class_num), #(10, 784) 정규분포난수
                #'w': cp.random.randn(X.shape[1], self.class_num),
                'b': np.random.randn(1, self.class_num),
                #'b': cp.random.randn(1, self.class_num),
                'w_': np.random.randn(X.shape[1], self.class_num),
                #'w_': cp.random.randn(X.shape[1], self.class_num),
                'b_': np.random.randn(1, self.class_num),
                #'b_': cp.random.randn(1, self.class_num),
                'tmpw': 0,
                'tmpb': 0
            }
        cnt=1
        if eval_score is None:
            self.score_val = 0
            
        for Xi in range(self.epochs):
            s_data, s_labels = self.shuffling(X, y)
            encoded_y=self.encoding(s_labels)
            avg_loss = 0
            batch_count = int(X.shape[0] / self.batch_size)
            for t in range(int(batch_count)):
                batch_X, batch_y, bs=self.batching(s_data, encoded_y, t)
                batch_X = np.reshape(batch_X, (bs, X.shape[1]))
                #batch_X = cp.reshape(batch_X, (bs, X.shape[1]))
                batch_y = np.reshape(batch_y, (bs, self.class_num))
                #batch_y = cp.reshape(batch_y, (bs, self.class_num))
                z = self.net_input(batch_X)
                loss = self.hinge_loss(batch_y, z)
                self.update_w_b(batch_X, batch_y, z, bs, cnt)
                cnt+=1
                avg_loss += loss
                self.update_count += 1
                
            self.params['tmpw'] = (cnt * (cnt/(cnt+1)) * 
                                   self.params['w_'] + (1/(cnt+1))*self.params['w'])
            self.params['tmpb'] = (cnt * (cnt/(cnt+1)) * 
                                   self.params['b_'] + (1/(cnt+1))*self.params['b'])
            prev_score = self.score_val
            pres_score = self.score(X, y)
            print("epochs: ", Xi)
            print("prev_score: ", prev_score)
            print("pres_score: ", pres_score,"\n")
            if prev_score < pres_score:
                self.score_val = pres_score
            if self.det_weight(X, y, 1) < self.det_weight(X, y): # temp_w, temp_b
                self.params['w_'] = self.params['tmpw']
                self.params['b_'] = self.params['tmpb']
            avg_loss /= batch_count
        return self
    
    def det_weight(self, X, y, aver=0):
        if aver:
            w1 = self.params['w_']
            b1 = self.params['b_']
        else:
            w1 = self.params['tmpw']
            b1 = self.params['tmpb']
        temp = np.dot(X, w1) + b1
        #temp = cp.dot(X, w1) + b1
        
        pred = np.argmax(temp, axis=1)
        #pred = cp.argmax(temp, axis=1)
        sco = np.mean(pred == y)
        #sco = cp.mean(pred == y)
        return sco
    
    def update_w_b(self, batch_X, batch_y, z, bs, cnt):
        n = np.shape(batch_X)[1]  # num of features
        #n = cp.shape(batch_X)[1] 
        delta_w = np.zeros(self.params['w'].shape)
        #delta_w = cp.zeros(self.params['w'].shape)
        delta_b = np.zeros(self.params['b'].shape)
        #delta_b = cp.zeros(self.params['b'].shape)
        z = np.reshape(z, (bs, self.class_num))
        #z = cp.reshape(z, (bs, self.class_num))
        temp = 1 - np.multiply(batch_y, z)
        #temp = 1 - cp.multiply(batch_y, z)
        temp[temp <= 0] = 0
        temp[temp > 0] = 1
        y_temp = np.multiply(batch_y, temp.reshape(bs, self.class_num))
        #y_temp = cp.multiply(batch_y, temp.reshape(bs, self.class_num))
        delta_w = -(1 / bs) * np.matmul(batch_X.T, y_temp) + (1 / self.C) * self.params['w']
        #delta_w = -(1 / bs) * cp.matmul(batch_X.T, y_temp) + (1 / self.C) * self.params['w']
        delta_b = -(1 / bs) * np.sum(y_temp, axis=0)
        #delta_b = -(1 / bs) * cp.sum(y_temp, axis=0)
        self.params['w'] = self.params['w'] - (self.eta / (1 + self.epsilon * cnt)) * delta_w
        self.params['b'] = self.params['b'] - (self.eta / (1 + self.epsilon * cnt)) * delta_b
        
        return self.params
    
    def hinge_loss(self, y, z):
        loss = 1 - np.multiply(y, z)
        #loss = 1 - cp.multiply(y, z)
        loss[loss < 0] = 0
        loss = np.mean(loss)
        #loss = cp.mean(loss)
        return loss
    
    def net_input(self, X):  # net_input() = forward_prop(), generate z
        z = np.matmul(X, self.params['w']) + self.params['b']
        #z = cp.matmul(X, self.params['w']) + self.params['b']
        return z

    def encoding(self, y):
        y_array=np.array(y).reshape(-1,1)

        enc=OneHotEncoder()
        enc.fit(y_array)
        y_enc=enc.transform(y_array).toarray()
        
        encoded_y=-1*np.ones((np.shape(y)[0],10))
        
        y_enc=y_enc * 2 + encoded_y
        
        """
        encoded_y=-1*np.ones((np.shape(y)[0],self.class_num))
        #encoded_y=-1*cp.ones((cp.shape(y)[0],self.class_num))
        for i in range(np.shape(y)[0]):
            encoded_y[i,y[i]] = 1
        """
        return y_enc
                
    def shuffling(self, X, y):
        temp_s=list(zip(X,y))
        random.shuffle(temp_s)
        X,y=zip(*temp_s)
        return X,y
    
    def batching(self, X, y, t):                         
        batch_X = X[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        batch_y = y[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        last_size = min(len(X), (t+1) * self.batch_size) - t * self.batch_size
        
        return batch_X, batch_y,last_size
    
    def predict(self, X):
        m = np.shape(X)[0]
        #m = cp.shape(X)[0]
        class_score = self.net_input(X)  # return z
        pred = np.argmax(class_score, axis=1)
        #pred = cp.argmax(class_score, axis=1)

        return pred
    
    def score(self, X, y):
        pred = self.predict(X)
        score = np.mean(pred == y)
        #score = cp.mean(pred == y)
        
        return score
    
    def get_params(self, deep=True):
        return {'C':self.C, 'batch_size':self.batch_size, 'epochs':self.epochs,
               'eta': self.eta, 'w':self.params['w_'], 'b':self.params['b_']}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def test(self, X, w, b):
        #z = np.matmul(X, np.array(w)) + np.array(b)
        z = cp.matmul(X, cp.array(w)) + cp.array(b)
        #p = np.argmax(z, axis=1)
        p = cp.argmax(z, axis=1)
        return p

In [ ]:
mine=myClassifier()
#mine.fit(X_train, y_train)
mine.fit(X_test,y_test)

fit params=None


/home/ryu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


epochs:  0
prev_score:  0
pres_score:  0.166 



/home/ryu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


epochs:  1
prev_score:  0.166
pres_score:  0.3492 



...

epochs:  197
prev_score:  0.9468
pres_score:  0.9469 



/home/ryu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


epochs:  198
prev_score:  0.9469
pres_score:  0.9469 



/home/ryu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


epochs:  199
prev_score:  0.9469
pres_score:  0.9466 



In [7]:
mine.get_params()

{'C': 1000,
 'batch_size': 60,
 'epochs': 200,
 'eta': 0.01,
 'w': array([[ 1.0854312 ,  1.60474952,  1.29690399, ...,  0.1711392 ,
          0.93949008,  1.25477187],
        [-0.51138963, -0.40311992,  0.94269138, ...,  0.14857752,
          0.5367502 ,  0.86845323],
        [ 0.6607393 ,  0.15685686,  1.41669104, ..., -0.02112389,
         -0.46241083, -0.72440239],
        ...,
        [-1.00923905,  0.33886948,  0.4260792 , ..., -0.29764807,
         -0.22932422, -0.34106951],
        [ 1.97990312,  1.62949041,  2.11518993, ..., -0.069971  ,
         -0.61228839,  0.55258546],
        [-0.46404495, -1.02294704, -1.02440946, ...,  1.44857499,
         -0.57983122,  0.19809106]]),
 'b': array([[ 0.78105322, -0.61673477,  1.04131276,  0.77108051, -0.22672777,
         -2.02352649,  0.71710939, -1.70611866,  0.24443303,  0.33781992]])}

In [8]:
w=mine.get_params()['w']
b=mine.get_params()['b']

In [9]:
p=mine.test(X_testall,w,b)

file=open('/home/ryu/AI/MyClassifier/prediction.txt','w')
for i in range(len(p)):
    file.write('%s\n' %p[i])
file.close()